# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!if [ ! -d flower_data ] ; then wget https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip; unzip -qq flower_data.zip;fi
!cp -f "/content/drive/My Drive/Colab Notebooks/flower_data/cat_to_name.json" /content/flower_data/cat_to_name.json
#!mkdir -p /content/flower_data
!ls -al /content/flower_data/
#!rsync -a "/content/drive/My Drive/Colab Notebooks/flower_data/" /content/flower_data/
!pwd
!df -h

!python -V
!pip3 install torch
!pip3 install torchvision
import torch
print(torch.version.cuda)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 20
drwx------   4 root root 4096 Jan  4 13:04 .
drwxr-xr-x   1 root root 4096 Jan  4 13:04 ..
-rw-------   1 root root 2218 Jan  4 13:20 cat_to_name.json
drwxr-xr-x 104 root root 4096 Jan 15  2018 train
drwxr-xr-x 104 root root 4096 Jan 15  2018 valid
/content
Filesystem      Size  Used Avail Use% Mounted on
overlay         359G   20G  321G   6% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
/dev/sda1       365G   24G  342G   7% /opt/bin
tmpfs           6.4G  8.0K  6.4G   1% /var/colab
shm             6.0G     0  6.0G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           119G  8.2G  111G   7% /content/drive
Python 3.6.7
9.0.176


In [3]:
!ls -al /content/flower_data/

# !rm -f /content/flower_data/cat_to_name.json

#!pip3 install PIL
#!python3 -c "import PIL;print(PIL.__version__); print(PIL); print(PIL.Image)"
# !pip3 freeze

total 20
drwx------   4 root root 4096 Jan  4 13:04 .
drwxr-xr-x   1 root root 4096 Jan  4 13:04 ..
-rw-------   1 root root 2218 Jan  4 13:20 cat_to_name.json
drwxr-xr-x 104 root root 4096 Jan 15  2018 train
drwxr-xr-x 104 root root 4096 Jan 15  2018 valid


In [4]:
# Imports here
import torch
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

from torchvision import datasets, transforms

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [0]:
# data_dir = '/content/flower_data'
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
from torchvision import datasets
import torchvision.transforms as transforms
# from torch.utils.data.sampler import SubsetRandomSampler

# TODO: Define your transforms for the training and validation sets
train_data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
valid_data_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# TODO: Load the datasets with ImageFolder
train_image_datasets = datasets.ImageFolder(train_dir, transform=train_data_transforms)
valid_image_datasets = datasets.ImageFolder(valid_dir, transform=valid_data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_dataloaders = torch.utils.data.DataLoader(train_image_datasets, batch_size=32, shuffle=True)
valid_dataloaders = torch.utils.data.DataLoader(valid_image_datasets, batch_size=32)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

In [8]:
# obtain one batch of training images
dataiter = iter(train_dataloaders)
# images, labels = dataiter.next()
# imshow(images[0])

# images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
# display 0 images
for idx in np.arange(0):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(labels[idx])

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [0]:
import json

# with open('cat_to_name.json', 'r') as f:
with open('/content/flower_data/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [11]:
from torchvision import datasets, transforms, models
from collections import OrderedDict

# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0003)

model.to(device);

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 68621284.09it/s]


In [12]:
# number of epochs to train the model
n_epochs = 10

valid_loss_min = np.Inf # track change in validation loss


for epoch in range(1, n_epochs+1):
    !date
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloaders):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        #data, target = data.to(device), target.to(device)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_dataloaders):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        #data, target = data.to(device), target.to(device)
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_dataloaders.dataset)
    valid_loss = valid_loss/len(valid_dataloaders.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

Fri Jan  4 13:20:57 UTC 2019
Epoch: 1 	Training Loss: 3.180537 	Validation Loss: 1.700567
Validation loss decreased (inf --> 1.700567).  Saving model ...
Fri Jan  4 13:24:22 UTC 2019
Epoch: 2 	Training Loss: 1.195161 	Validation Loss: 0.781435
Validation loss decreased (1.700567 --> 0.781435).  Saving model ...
Fri Jan  4 13:27:49 UTC 2019
Epoch: 3 	Training Loss: 0.633744 	Validation Loss: 0.495987
Validation loss decreased (0.781435 --> 0.495987).  Saving model ...
Fri Jan  4 13:31:15 UTC 2019
Epoch: 4 	Training Loss: 0.421018 	Validation Loss: 0.376554
Validation loss decreased (0.495987 --> 0.376554).  Saving model ...
Fri Jan  4 13:34:40 UTC 2019
Epoch: 5 	Training Loss: 0.312354 	Validation Loss: 0.316525
Validation loss decreased (0.376554 --> 0.316525).  Saving model ...
Fri Jan  4 13:38:05 UTC 2019
Epoch: 6 	Training Loss: 0.256367 	Validation Loss: 0.297436
Validation loss decreased (0.316525 --> 0.297436).  Saving model ...
Fri Jan  4 13:41:31 UTC 2019
Epoch: 7 	Training Los

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
# TODO: Save the checkpoint 
torch.save(model.state_dict(), 'model.pt')

In [14]:
!rsync -P /content/model.pt "/content/drive/My Drive/Colab Notebooks/"

model.pt
     30,559,781 100%  143.41MB/s    0:00:00 (xfr#1, to-chk=0/1)


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [15]:
!ls -al

total 334016
drwxr-xr-x 1 root root      4096 Jan  4 13:24 .
drwxr-xr-x 1 root root      4096 Jan  4 13:03 ..
drwxr-xr-x 1 root root      4096 Dec 18 20:29 .config
drwx------ 3 root root      4096 Jan  4 13:04 drive
drwx------ 4 root root      4096 Jan  4 13:04 flower_data
-rw-r--r-- 1 root root 311442766 Nov  6 16:00 flower_data.zip
-rw-r--r-- 1 root root  30559781 Jan  4 13:55 model.pt
drwxr-xr-x 1 root root      4096 Dec 18 20:29 sample_data


In [0]:
#!rsync -P "/content/drive/My Drive/Colab Notebooks/model.pt" /content/model.pt

In [17]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
import torch
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from collections import OrderedDict

# TODO: switch to gpu
state_dict = torch.load('model.pt', map_location='cpu')
model = models.densenet121(pretrained=True)
model.classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

model.load_state_dict(state_dict)
print(model)

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [0]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

    # large images will slow down processing
    in_transform = transforms.Compose([
                        transforms.Resize(255),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])

    # discard the transparent, alpha channel (that's the :3)
    image = in_transform(image)[:3,:,:]
    print(image.size())
    image = image.numpy()
    print(image.shape)
    return image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [0]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    image = image.transpose(1,2,0)
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [0]:
from PIL import Image

def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    image = Image.open(image_path).convert('RGB')
    data = process_image(image)
    print(data.shape)
    # imshow(data)
    b = torch.from_numpy(data)
    b = b.unsqueeze(0)
    output = model.forward(b)
    print(output)
    p = F.softmax(output, dim=1).data
    if(train_on_gpu):
        p = p.cpu() # move to cpu
    prob, indices = torch.topk(p, topk)
    prob = prob.numpy().squeeze()
    indices = indices.numpy().squeeze()
    return data, prob, indices
    
    

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [21]:
# TODO: Display an image along with the top 5 classes
import matplotlib.pyplot as plt

data, prob, classes = predict("flower_data/valid/1/image_06739.jpg", model)
# imshow(data)
print(prob)
print(classes)
classes = [cat_to_name[str(c)] for c in classes]
print(classes)

# plt.barh(classes, prob)

torch.Size([3, 224, 224])
(3, 224, 224)
(3, 224, 224)
tensor([[-3.5449, -9.9671, -6.3194, -4.1244, -7.9255, -6.4107, -6.5859, -6.9591,
         -8.1365, -4.8077, -7.7423, -6.9538, -4.5412, -6.5796, -6.7881, -5.9667,
         -5.3263, -5.5745, -7.4999, -5.9590, -5.1067, -3.8848, -7.4070, -6.8529,
         -7.9571, -4.8134, -3.2825, -5.3452, -3.7403, -7.2720, -6.1740, -9.2797,
         -3.6052, -7.9586, -4.2788, -6.3712, -3.8480, -4.6183, -4.6111, -5.9705,
         -4.3463, -6.9942, -5.2086, -5.5039, -8.4626, -6.3652, -7.9611, -5.6007,
         -8.7509, -2.2446, -5.9065, -3.7754, -7.2192, -5.2520, -5.7800, -5.3992,
         -9.4969, -5.7258, -5.5399, -6.2292, -5.9984, -2.8510, -9.1346, -7.0342,
         -5.6550, -8.9129, -5.0560, -5.0861, -7.5418, -7.1126, -6.2462, -7.9296,
         -2.9513, -6.4487, -5.3874, -7.2406, -6.6436, -4.0511, -6.0941, -6.6975,
         -7.4320, -5.8957, -6.3703, -3.4657, -3.2925, -3.2340, -4.7095, -4.2940,
         -5.9560, -5.4270, -3.0473, -5.7179, -3.2480, -